# 필요한 모듈 설치 및 임포트

In [1]:
pip install selenium==4.8.2


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
print(selenium.__version__)

4.8.2


# Selenium을 이용하여 브라우저 열기, 클릭, 페이지 이동, 닫기

In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\82107\AppData\Local\Temp\ipykernel_22848\2160715986.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [4]:
url = 'https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1'
driver.get(url)
driver.maximize_window() # 브라우저의 크기 전체화면으로 확대

# 무신사 크롤링

In [25]:
# options = webdriver.ChromeOptions()
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver = webdriver.Chrome(ChromeDriverManager().install())
# driver = webdriver.Chrome(options=chrome_options)

# 무신사 코디맵 1페이지(많은 댓글 수 기준 정렬)에 나와있는 60개 코디맵에 대하여 
# 각각 [‘codimap_category’, ‘codimap_title’, ‘codimap_date’, ‘views’, comment_numbers’] 
# 정보를 가져와서 csv형태로 만든 다음, 해당 코드(.ipynb)와 csv파일을 제출

codimap_list = []

url = 'https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1'
driver.get(url)
driver.implicitly_wait(2)
try:
    driver.maximize_window()
except:
    pass

# 페이지를 가장 아래까지 스크롤하는 방법

interval = 1 # 1초에 한번씩 스크롤 내림

# 현재 문서 높이를 가져와서 저장
prev_height = driver.execute_script("return document.body.scrollHeight")

# 반복 수행
while True:
    # 스크롤을 가장 아래로 내림
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

    # 페이지 로딩 대기
    time.sleep(interval)

    # 현재 문서 높이를 가져와서 저장
    curr_height = driver.execute_script("return document.body.scrollHeight")
    if curr_height == prev_height:
        break

    prev_height = curr_height

# 해당 Xpath 정보를 확인할때까지 5초간 기다림, 5초후에도 해당 Xpath를 찾지 못하면 에러 반환
# elem = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div[1]/ul/li[40]/a/div/img')))

columns = ['codimap_category', 'codimap_title', 'codimap_date', 'views', 'comment_numbers']
values = []
soup = BeautifulSoup(driver.page_source, 'lxml')
data_rows = soup.find_all('li', attrs={'class':'style-list-item'}) # 해당 조건에 맞는 정보들을 list로 반환

for i, row in enumerate(data_rows):

    print('{0}번째 코디맵 크롤링'.format(i+1))
    blank = []

    # 코디맵 카테고리 가져오기
    codimap_category = row.find('span', attrs={'class':'style-list-information__text'})
    if codimap_category:
        codimap_category = codimap_category.get_text().strip()
        blank.append(codimap_category)
    else:
        blank.append('Something is wrong')
        print('{}번째 카테고리 가져올때 문제발생'.format(i+1))
        continue

    # 코디맵 제목 가져오기
    codimap_title = row.find('strong', attrs={'class':'style-list-information__title'})
    if codimap_title:
        codimap_title = codimap_title.get_text().strip()
        blank.append(codimap_title)
    else:
        blank.append('Something is wrong')
        print('{}번째 제목 가져올때 문제발생'.format(i+1))
        continue

    # 코디맵 날짜, 조회수, 댓글 개수 가져오기
    post_info_set = row.find_all('span', attrs={'class':'post-information__text'})
    if post_info_set:
        codimap_date = post_info_set[0].get_text().strip()
        views = post_info_set[1].get_text().strip().split(" ")[1]
        comment_numbers = post_info_set[2].get_text().strip().split(" ")[1]
        
        blank.append(codimap_date)
        blank.append(views)
        blank.append(comment_numbers)
    else:
        blank.append('Something is wrong')
        print('{}번째 날짜, 조회수, 댓글 개수 가져올때 문제발생'.format(i+1))
        continue

    values.append(blank)
    print('---------------------------------------------------')

df = pd.DataFrame(values, columns = columns)
codimap_list.append(df) # 최종적으로 [[토요일 웹툰 정보 리스트들], [일요을 웹툰 정보 리스트들]] 형태로 반환됨


C:\Users\82107\AppData\Local\Temp\ipykernel_22848\188055731.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


1번째 코디맵 크롤링
---------------------------------------------------
2번째 코디맵 크롤링
---------------------------------------------------
3번째 코디맵 크롤링
---------------------------------------------------
4번째 코디맵 크롤링
---------------------------------------------------
5번째 코디맵 크롤링
---------------------------------------------------
6번째 코디맵 크롤링
---------------------------------------------------
7번째 코디맵 크롤링
---------------------------------------------------
8번째 코디맵 크롤링
---------------------------------------------------
9번째 코디맵 크롤링
---------------------------------------------------
10번째 코디맵 크롤링
---------------------------------------------------
11번째 코디맵 크롤링
---------------------------------------------------
12번째 코디맵 크롤링
---------------------------------------------------
13번째 코디맵 크롤링
---------------------------------------------------
14번째 코디맵 크롤링
---------------------------------------------------
15번째 코디맵 크롤링
---------------------------------------------------
16번째 코디맵 크롤링
---------------------

In [26]:
df_ = pd.concat(codimap_list).reset_index(drop=True) # index 초기화 해줘야함
df_.head(10)

,codimap_category,codimap_title,codimap_date,views,comment_numbers
0,캐주얼,톤온톤 코디,21.07.02,"17,216",50
1,캐주얼,블랙 매니아,21.02.23,"36,276",47
2,캐주얼,오늘의 코디,22.03.22,"13,365",43
3,댄디,추천 댄디 룩,21.09.15,"21,567",32
4,캐주얼,릴랙스 룩,21.02.23,"24,003",25
5,스트릿,힙하다 힙해,21.02.03,"36,048",24
6,댄디,데이트 코디,21.12.24,"10,945",19
7,캐주얼,심플한 코디,21.07.30,"41,333",18
8,댄디,훈훈 남친 룩,21.03.24,"19,976",17
9,댄디,서머 데이트 룩,21.07.26,"8,349",17


In [27]:
print(df_.shape)

(60, 5)


# Dataframe을 csv 파일로 저장

In [28]:
df_.to_csv('무신사코디맵.csv', encoding = 'utf-8-sig') # 한글이 들어간 정보는 글자가 깨질 수 있어 encoding을 다음과 같이 지정하여 저장